In [33]:
import pandas as pd
import os
import csv

# Define the path to your data directory
data_directory = "./data"
file_names = [
    file
    for file in os.listdir(data_directory)
    if all([file.endswith(".csv"), "concatenated_autotrader.csv" not in file, "transformed" not in file])
]

# !EXTRACT

# Import all CSV files as a list of dataframes
dataframes = []
for file_name in file_names:
    file_path = os.path.join(data_directory, file_name)
    try:
        df = pd.read_csv(file_path)
        dataframes.append(df)
        headers_list = list(df.columns)
        first_row_list = list(df.iloc[0])
                # Writing the lists to a CSV file
        with open('lists.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(headers_list)
            writer.writerow( first_row_list)

    except Exception as e:
        print(f"Error reading {file_name}: {e}")

# Concatenate all dataframes into one
concatenated_df = pd.concat(dataframes, ignore_index=True)

# Optional: Save the concatenated dataframe to a new CSV file
concatenated_df.to_csv(
    os.path.join(data_directory, "concatenated_autotrader.csv"), index=False
)

print("All datasets have been concatenated into one dataframe.")


All datasets have been concatenated into one dataframe.


In [38]:
import pandas as pd
import re 

# Step 1: Load the concatenated CSV file
df = pd.read_csv('./data/concatenated_autotrader.csv')

# !TRANSFORM

# Step 2: Manually mapping current headers to desired headers
# This is an example based on the given first row of data and desired headers.
# You'll need to adjust these based on the actual data in your file.
header_mapping = {
    'sc-kpDqfm 2': 'Make',
    'sc-jlZhew': 'Model',
    'sc-kpDqfm href': 'Title_URL',
    'sc-jEACwC 2': 'Sale_Type',  # Example guess, adjust based on actual data inspection
    'at__sc-1mc7cl3-5': 'Price',
    'sc-jlZhew 2': 'Additional_Comments',
    'sc-jlZhew 3': 'Year',
    'sc-jlZhew 4': 'Car_Type',
    'sc-jlZhew 5': 'Mileage',
    'sc-jlZhew 6': 'Engine_Size',
    'sc-jlZhew 7': 'Engine_Type',
    'sc-jlZhew 8': 'Transmission',
    'sc-jlZhew 9': 'Dealership_Location',
    'sc-jlZhew 10': 'Prior_Owners',
}

# Renaming the columns based on the mapping
df_renamed = df.rename(columns=header_mapping)

# Select only the columns we want to keep, dropping any that aren't listed in the mapping
df_transformed = df_renamed[list(header_mapping.values())]



# Transform the Price column from "£40,750" to an integer, handling missing values
df_transformed['Price'] = df_transformed['Price'].str.replace('£', '').str.replace(',', '')
df_transformed['Price'] = pd.to_numeric(df_transformed['Price'], errors='coerce')

# Use a regular expression to remove any content within parentheses and any non-numeric characters, then convert to numeric
df_transformed['Year'] = df_transformed['Year'].astype(str).apply(lambda x: re.sub(r'\s*\(.*\)', '', x))
df_transformed['Year'] = df_transformed['Year'].str.extract('(\d{4})')  # Extracts only the year (4 digits)
df_transformed['Year'] = pd.to_numeric(df_transformed['Year'], errors='coerce')

df_transformed['Mileage'] = df_transformed['Mileage'].str.replace('miles', '')
df_transformed['Mileage'] = pd.to_numeric(df_transformed['Mileage'], errors='coerce')

df_transformed['Prior_Owners'] = df_transformed['Prior_Owners'].str.replace('owner', '').replace('s', '')
df_transformed['Prior_Owners'] = pd.to_numeric(df_transformed['Prior_Owners'], errors='coerce')

# Fill in missing data with 'Unknown' only for Engine_Size column
df_transformed['Engine_Size'] = df_transformed['Engine_Size'].fillna('Unknown')

# Reorder columns to match the requested sequence
columns_order = [
    'Make', 'Title_URL', 'Sale_Type', 'Price', 'Model', 'Additional_Comments',
    'Year', 'Car_Type', 'Mileage', 'Engine_Size', 'Engine_Type',
    'Transmission', 'Dealership_Location', 'Prior_Owners'
]
df_transformed = df_transformed[columns_order]


# Fill in missing data with 'Unknown' for other categorical columns
categorical_columns = ['Model', 'Additional_Comments', 'Car_Type', 'Engine_Type', 'Dealership_Location', 'Sale_Type']
df_transformed[categorical_columns] = df_transformed[categorical_columns].fillna('Unknown')

# Remove rows with 'Unknown' values in critical columns
df_transformed = df_transformed[df_transformed['Price'] != 'Unknown']
df_transformed = df_transformed[df_transformed['Mileage'] != 'Unknown']
df_transformed = df_transformed[df_transformed['Year'] != 'Unknown']

# Adding the delete duplicates feature
cleaned_df = df_transformed.drop_duplicates()

# Load additional data
additional_file_path = 'data/legacy/cleaned_auto_trader_data_aston_martin.csv'
additional_df = pd.read_csv(additional_file_path)

# Check for header alignment
current_headers = set(cleaned_df.columns)
additional_headers = set(additional_df.columns)
unmatched_headers = current_headers.symmetric_difference(additional_headers)

if unmatched_headers:
    print("Headers that don't line up:", unmatched_headers)
else:
    print("All headers line up perfectly.")

# Concatenate data from both DataFrames where the headers line up
# Align additional_df to cleaned_df's columns to ensure consistency
aligned_additional_df = additional_df[sorted(list(current_headers.intersection(additional_headers)))]
concatenated_df = pd.concat([cleaned_df, aligned_additional_df], ignore_index=True)

# Save the concatenated and cleaned data to a new file
concatenated_file_path = 'data/production/autotrader.csv'
concatenated_df.to_csv(concatenated_file_path, index=False)

print("The transformed and concatenated data has been saved to 'data/transformed_autotrader.csv'.")

Headers that don't line up: {'Information/Detail'}
The transformed and concatenated data has been saved to 'data/transformed_autotrader.csv'.


/var/folders/1f/kh62j3491pg82ts5mywnf4hc0000gn/T/ipykernel_33982/2394408829.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transformed['Price'] = df_transformed['Price'].str.replace('£', '').str.replace(',', '')
/var/folders/1f/kh62j3491pg82ts5mywnf4hc0000gn/T/ipykernel_33982/2394408829.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transformed['Price'] = pd.to_numeric(df_transformed['Price'], errors='coerce')
/var/folders/1f/kh62j3491pg82ts5mywnf4hc0000gn/T/ipykernel_33982/2394408829.py:4